In [4]:
import polars as pl
import numpy as np
from sklearn.metrics.pairwise import haversine_distances
import os
import pandas as pd
import matplotlib.pyplot as plt
import haversine as hs
import gc
os.getcwd()

'c:\\Users\\digom\\Documents\\GitHub\\fh-energia'

In [5]:
df_cnpj = pl.scan_csv('./data/estabelecimentos/estabelecimentos.csv', has_header=False, infer_schema_length=0)


In [6]:
situ_dict = {'01': 'Nula', '02': 'Ativa', '03': 'Suspensa', '04': 'Inapta', '08': 'Baixada'}

df_cnpj = df_cnpj.select('column_1', 'column_2', 'column_3', 'column_4', 'column_5', 'column_6',
                         'column_11',  'column_12', 'column_13',
                          'column_14', 'column_15', 'column_16', 'column_19', 'column_20', 'column_21')
df_cnpj = df_cnpj.with_columns(pl.concat_str(
        [
            pl.col("column_1"),
            pl.col("column_2"),
            pl.col("column_3"),
        ],
        separator="",
    ).alias('cnpj'),
    pl.concat_str(
        [
            pl.col("column_14"),
            pl.col("column_15"),
            pl.col("column_16"),
        ],
        separator=" ",
    ).alias('endereco'),
    pl.col('column_6').replace(situ_dict).alias('situacao_cadastral'))\
            .rename({'column_4': 'matriz_filial',  'column_19': 'CEP', 'column_12':'CNAE', 'column_13':'CNAE_2', 'column_16': 'numero', 'column_5': 'nome_fantasia',
                     'column_20': 'UF', 'column_21': 'municipio', 'column_11':'CNPJ_DAT_CON'})\
            .with_columns(cnae_2_split=pl.col("CNAE_2").str.split(","))\
            .with_columns(pl.col('CNPJ_DAT_CON').str.to_date('%Y%m%d').alias('CNPJ_DAT_CON'))\
            .with_columns(
                          pl.col('CNPJ_DAT_CON').dt.month().alias('CNPJ_DAT_CON_M'),
                          pl.col('CNPJ_DAT_CON').dt.year().alias('CNPJ_DAT_CON_Y'))\
            .explode('cnae_2_split')\
            .select(['cnpj', 'endereco', 'matriz_filial', 'CEP','CNAE', 'cnae_2_split', 'numero', 'situacao_cadastral', 
                     'nome_fantasia', 'UF', 'municipio', 'CNPJ_DAT_CON', 'CNPJ_DAT_CON_M', 'CNPJ_DAT_CON_Y'])\
            .filter(pl.col('situacao_cadastral').is_in(['Ativa', 'Suspensa', 'Inapta']))

In [7]:
df_bdgd = pl.scan_csv('data/bdgd/ucat_pj.csv', separator=";", encoding='utf8-lossy', infer_schema_length=0)
df_bdgd = df_bdgd.with_columns(pl.col('CEP').str.replace('\.', '').str.replace('-', '').alias('CEP'),
                                pl.col('CNAE').str.replace('-', '').str.replace('/', '').alias('CNAE'),
                                pl.col('DAT_CON').str.to_date('%d/%m/%Y').alias('DAT_CON'),)\
                 .with_columns(
                             pl.col('DAT_CON').dt.month().alias('DAT_CON_M'),
                             pl.col('DAT_CON').dt.year().alias('DAT_CON_Y'))\
                 .with_row_index()


In [9]:

df_cnae_1 = df_cnpj.join(df_bdgd, on=['CEP', 'CNAE'], how='inner')
#df_cnae_2 = df_cnpj.join(df_bdgd, left_on=['CEP', 'cnae_2_split'], right_on=['CEP', 'CNAE'], how='inner')
#concatenating the two dataframes
df_final = df_cnae_1#pl.concat([df_cnae_1, df_cnae_2])
df_filter = df_final.unique(subset=['COD_ID', 'cnpj'])
df_cep_cnae_num = df_filter#.collect()#.to_pandas()

In [ ]:
#            .filter(pl.col('LGRD').str.contains(pl.col('numero'), literal=True))\
                    

In [12]:
df_cep_cnae_num.select(pl.len()).collect()

len
u32
2353


In [13]:
df_cep_cnae_num_pandas = df_cep_cnae_num.collect().to_pandas()

In [17]:
df_cep_cnae_num_pandas.to_csv('data/whole_data/ucat_join.csv', index=False)

In [16]:
df_cep_cnae_num_pandas
#count cnpj per COD_ID
df_count = df_cep_cnae_num_pandas.groupby('COD_ID')['cnpj'].count().reset_index()
df_count

,COD_ID,cnpj
0,0000085014,1
1,0000598682,1
2,0001159933,1
3,0001681656,13
4,0001761439,1
...,...,...
631,ATE0011635,1
632,ATE0013234,1
633,ATE0015199,2
634,ATE0018600,4


In [128]:
len(df_cep_cnae_num_pandas['cnpj'].value_counts().loc[lambda x: x == 1].index)

215

In [132]:
len(df_cep_cnae_num_pandas['COD_ID'].value_counts().loc[lambda x: x == 1].index)

69

In [130]:
single_cnpj = df_cep_cnae_num_pandas['cnpj'].value_counts().loc[lambda x: x==1].index
df_merge_single_cnpj = df_cep_cnae_num_pandas.loc[df_cep_cnae_num_pandas['cnpj'].isin(single_cnpj)]
df_merge_single_cnpj

,cnpj,endereco,matriz_filial,CEP,CNAE,cnae_2_split,numero,situacao_cadastral,nome_fantasia,UF,...,FIC_12,DESCR,POINT_X,POINT_Y,DAT_CON_M,DAT_CON_Y,LATITUDE,LONGITUDE,DISTANCE_IBGE,NEAREST_IBGE
1,18788137000207,RODOVIA BRIGADEIRO FARIA LIMA S/N,2,14840000,3511501,3513100,S/N,Ativa,None,SP,...,0,None,-48.28381573,-21.41425121,4,2020,-21.418970,-48.291576,0.959500,1565644
2,30572273000171,RUA Padre Pedro Henrique 02,1,12570000,4755502,1749400,02,Inapta,None,SP,...,0,A001199588,-45.22556449,-22.83775849,10,1986,-22.837781,-45.225171,0.040402,9737652
4,08184109000138,"ESTRADA MUNICIPAL BOITUVA A TATUI, KM 3,5 S/N",1,18550000,0155501,0113000,S/N,Ativa,None,SP,...,0,None,-47.76335417,-23.29719573,12,2015,-23.296917,-47.764368,0.108081,19936331
5,00394502049151,"RODOVIA SOROCABA / IPERO KM 12,5 S/N",2,18560000,8422100,8411600,S/N,Ativa,BTLDEFNBQR-ARAMAR,SP,...,0,None,-47.59769689,-23.40154314,2,2009,-23.402507,-47.589477,0.845645,9293312
6,07689002000421,ESTRADA MUNICIPAL EUCLIDES MARTINS 2170,2,14813000,3041500,3042300,2170,Ativa,None,SP,...,0,None,-48.39648599,-21.76170142,4,2012,-21.760875,-48.398255,0.204497,14507411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,02044526000107,AVENIDA ANTONIO FADIN 751,1,13147030,4681801,4684299,751,Ativa,SMALL,SP,...,0,None,-47.13300639,-22.73431915,6,2018,-22.736492,-47.139204,0.679970,9220832
322,21530705000190,ESTRADA SITIO GRANDE S/N1 S/N,1,18550000,0151202,0155501,S/N,Ativa,None,SP,...,0,None,-47.76335417,-23.29719573,12,2015,-23.296917,-47.764368,0.108081,19936331
323,08070508012265,RUA DO ENGENHO S/N,2,13370000,1071600,3511501,S/N,Ativa,None,SP,...,0,None,-47.52980746,-23.00813344,11,2017,-23.008206,-47.529642,0.018759,18369476
324,61064838001296,AVENIDA DO CRISTAL 530,2,12311210,2229303,2223400,530,Ativa,None,SP,...,0,A001199599,-45.93612378,-23.28810537,7,2003,-23.288571,-45.932507,0.373013,14452084


In [133]:
#melhores cnpj
df_merge_single_cnpj
best_cnpj = df_merge_single_cnpj['COD_ID'].value_counts().loc[lambda x: x==1].index
df_best_cnpj = df_merge_single_cnpj.loc[df_merge_single_cnpj['COD_ID'].isin(best_cnpj)].drop_duplicates(subset=['cnpj'])
df_best_cnpj

,cnpj,endereco,matriz_filial,CEP,CNAE,cnae_2_split,numero,situacao_cadastral,nome_fantasia,UF,...,FIC_12,DESCR,POINT_X,POINT_Y,DAT_CON_M,DAT_CON_Y,LATITUDE,LONGITUDE,DISTANCE_IBGE,NEAREST_IBGE
12,53408860000125,FAZENDA SANTA MARIA SN,1,19904565,1071600,1099603,SN,Ativa,None,SP,...,0,None,-49.77842262,-22.94224343,11,2016,-22.944307,-49.770162,0.876446,9528147
17,49418890000145,AVENIDA SAUDADE 197,1,13260000,1313800,6491300,197,Ativa,None,SP,...,0,None,-46.79976496,-22.87561523,5,2015,-22.875555,-46.800907,0.117193,10922560
18,56431364000180,AVENIDA DOUTOR AUGUSTO APARECIDO ARROYO MARCHI...,1,15507000,1012101,1013901,3.545,Ativa,FRIGORIFICO AVICOLA VOTUPORANGA,SP,...,0,None,-49.99694535,-20.40944426,3,1993,-20.409419,-50.001766,0.502391,18465013
19,13743550002862,RUA SEBASTIAO CARDOSO (LOTEAMENTO JARDIM OKINA...,2,13141005,7733100,6190699,S/N,Ativa,None,SP,...,0,None,-47.13490593,-22.79768897,7,2021,-22.799319,-47.133630,0.223515,9212518
24,15664292000134,RODOVIA SP-147 S/N,1,13414020,3811400,4687703,S/N,Ativa,None,SP,...,0,None,-47.52451987,-22.65699717,6,2013,-22.657327,-47.524318,0.042121,20147995
31,29774831000100,RUA TENENTE ADOLFO PADILHA 49,1,16880000,0161003,1066000,49,Ativa,MM QUEIROZ,SP,...,0,None,-50.95694041,-21.32873533,6,2010,-21.331891,-50.957362,0.353602,15751193
48,53309795000180,RUA QUARTORZE 1.126,1,14620000,2424501,2592601,1.126,Ativa,None,SP,...,0,None,-47.89700687,-20.71227428,6,2014,-20.712039,-47.899793,0.290960,12524164
61,46344354000154,AVENIDA ORIENTO S/N,1,18500000,2029100,2013402,S/N,Ativa,AJINOMOTO DO BRASIL,SP,...,0,None,-47.81980502,-23.06031892,8,1998,-23.060984,-47.817219,0.274716,13323053
62,01192333000122,ESTRADA MUNICIPAL VALENCIO CALEGARI 777,1,13181903,2910701,2543800,777,Ativa,None,SP,...,0,None,-47.19906037,-22.83414978,2,2009,-22.834915,-47.199662,0.105078,10860680
64,32794003000130,AVENIDA Avenida Jose Garcia Junqueira 237,1,14790000,4724500,1071600,237,Ativa,None,SP,...,0,None,-48.18641229,-20.27868022,7,2016,-20.277498,-48.191207,0.517093,11209022


In [116]:
single_cod_id = df_cep_cnae_num_pandas['COD_ID'].value_counts().loc[lambda x: x==1].index
df_merge_single_cod_id = df_cep_cnae_num_pandas.loc[df_cep_cnae_num_pandas['COD_ID'].isin(single_cod_id)].drop_duplicates(subset=['cnpj'])
df_merge_single_cod_id

,cnpj,endereco,matriz_filial,CEP,CNAE,cnae_2_split,numero,situacao_cadastral,nome_fantasia,UF,...,FIC_12,DESCR,POINT_X,POINT_Y,DAT_CON_M,DAT_CON_Y,LATITUDE,LONGITUDE,DISTANCE_IBGE,NEAREST_IBGE
4,02573802000116,RODOVIA ASSIS CHATEAUBRIAND S/N,1,15400000,1111901,4723700,S/N,Ativa,None,SP,...,0,None,-49.10938915,-20.68811229,5,2019,-20.686213,-49.104354,0.564758,11260020
6,52850682000478,RUA DAS PALMAS 84,2,15910000,2219600,8299799,84,Ativa,None,SP,...,0,None,-48.50014097,-21.27607412,2,2012,-21.273472,-48.500485,0.291531,16625840
7,08110543000173,"ESTRADA MUNICIPAL VPS 321, KM 22,8 S/N",1,16880000,1931400,1066000,S/N,Ativa,DA MATA S.A. - ACUCAR E ALCOOL,SP,...,0,None,-50.95694041,-21.32873533,6,2010,-21.331891,-50.957362,0.353602,15751193
9,15664292000134,RODOVIA SP-147 S/N,1,13414020,3811400,4687703,S/N,Ativa,None,SP,...,0,None,-47.52451987,-22.65699717,6,2013,-22.657327,-47.524318,0.042121,20147995
12,45281813000135,ESTRADA DA ANTIGA FAZ ITAQUERE S/N,1,14920000,1071600,1931400,S/N,Ativa,None,SP,...,0,None,-48.59393035,-21.76934678,5,2022,-21.765472,-48.604038,1.129211,8332004
13,53877627000949,AVENIDA SANTOS DUMONT 4444,2,11460902,2099199,2031200,4444,Ativa,None,SP,...,0,None,-46.27400911,-23.97636123,2,2000,-23.976356,-46.275493,0.150765,20996818
14,01576817000256,ESTRADA GIUSEPPINA VIANELLI DI NAPOLLI 600,2,13086530,7210000,8599699,600,Ativa,CNPEM FILIAL 01,SP,...,0,None,-47.05558489,-22.80701528,1,2020,-22.809967,-47.050060,0.654542,16865217
15,71586952001230,AVENIDA PRESIDENTE HUMBERTO DE ALENCAR CASTELO...,2,12321150,2424502,None,2705,Ativa,ARMCO FILIAL JACAREI,SP,...,0,A003506660,-45.98637208,-23.28917921,6,2013,-23.289088,-45.980035,0.647317,14359252
18,02476026000802,RODOVIA CONEGO DOMENICO RANGONI S/N,2,11096000,5231102,5212500,S/N,Ativa,None,SP,...,0,None,-46.36033795,-23.85827877,3,2016,-23.857653,-46.361278,0.118239,15873998
19,14675270000450,RUA RIBEIRAO PRETO 811,2,13323010,1621800,4671100,811,Ativa,ECTX S/A - EUCATEX,SP,...,0,None,-47.2800216,-23.20688094,8,2012,-23.207667,-47.279643,0.095587,1011723


In [22]:
df_merge_single_cnpj

,cnpj,endereco,matriz_filial,CEP,CNAE,cnae_2_split,numero,situacao_cadastral,nome_fantasia,UF,...,FIC_12,DESCR,POINT_X,POINT_Y,DAT_CON_M,DAT_CON_Y,LATITUDE,LONGITUDE,DISTANCE_IBGE,NEAREST_IBGE
0,05526979000150,AVENIDA ANDROMEDA 227,1,12230000,6810202,6822600,227,Ativa,None,SP,...,0,A001199627,-45.89485858,-23.2162918,1,1986,-23.216415,-45.892972,0.193277,21607
2,02998301001072,RODOVIA PREFEITO FABIO TALARICO S/N,2,14600000,3511501,None,S/N,Ativa,None,SP,...,0,None,-47.81359341,-20.54930676,11,2011,-20.546050,-47.815068,0.393339,3174411
3,08494345000231,SÍTIO SAKURA NAKAYA I S/N,2,18550000,0151201,0155501,S/N,Ativa,None,SP,...,0,None,-47.76335417,-23.29719573,12,2015,-23.296551,-47.762360,0.124293,2842465
4,14675270000450,RUA RIBEIRAO PRETO 811,2,13323010,1621800,4671100,811,Ativa,ECTX S/A - EUCATEX,SP,...,0,None,-47.2800216,-23.20688094,8,2012,-23.207731,-47.279441,0.111603,143108
6,40504325000983,RODOVIA JORNALISTA FRANCISCO AGUIRRE PROENCA S/N,2,13186904,4751201,None,S/N,Ativa,None,SP,...,0,None,-47.19918326,-22.89832035,4,2022,-22.898640,-47.198108,0.115735,3072755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,76490184003607,RODOVIA DOM GABRIEL PAULINO BUENO COUTO S/N,2,13212240,1033302,1122499,S/N,Ativa,None,SP,...,0,None,-46.95958493,-23.17361744,3,2019,-23.174023,-46.961757,0.226570,1652738
108,62695036004009,RUA FLORINDO CIBIN 7000,2,13470437,2029100,2019399,7000,Ativa,EVONIK,SP,...,0,None,-47.35484203,-22.7396094,5,2020,-22.739328,-47.354314,0.062541,2884900
109,13743550000142,AVENIDA JOAO BATISTA NUNES 50,1,13288162,7733100,6204000,50,Ativa,None,SP,...,0,None,-47.01297876,-23.07190598,6,2021,-23.070978,-47.010998,0.227394,2675009
110,52850682000478,RUA DAS PALMAS 84,2,15910000,2219600,8299799,84,Ativa,None,SP,...,0,None,-48.50014097,-21.27607412,2,2012,-21.273472,-48.500485,0.291531,2338768
